* 비동기 I/O 코루틴 개념
* Block I/O 순차 실행 예제
* ThreadPool 사용 우회
* Asyncio 실습 예제

# Asyncio
* 비동기 I/O Coroutine 작업
* Generator -> 반복적인 객체 Return 사용
* 즉, 실행 Stop -> 다른 작업으로 위임 -> Stop 지점 부터 재실행 원리
* non-blocking 비동기 처리에 적합

In [3]:
import timeit
from urllib.request import urlopen

urls = ['http://daum.net', 'https://google.com', 'https://apple.com', 'https://tistory.com', 'https://github.com/', 'https://gmarket.co.kr/']
start = timeit.default_timer()

# 순차 실행부
for url in urls:
    print('Start', url)
    urlopen(url)
    print('Done', url)

# 완료시간 - 시작시간
duration = timeit.default_timer() - start

# 총 실행 시간
print('Total Time : ', duration)

Start http://daum.net
Done http://daum.net
Start https://google.com
Done https://google.com
Start https://apple.com
Done https://apple.com
Start https://tistory.com
Done https://tistory.com
Start https://github.com/
Done https://github.com/
Start https://gmarket.co.kr/


HTTPError: HTTP Error 308: Permanent Redirect

In [1]:
import timeit
from urllib.request import urlopen
from concurrent.futures import ThreadPoolExecutor
import threading

# 시작 시간
start = timeit.default_timer()
urls = ['http://daum.net', 'https://google.com', 'https://apple.com', 'https://tistory.com', 'https://github.com/', 'https://gmarket.co.kr/']


def fetch(url):
    print('Thread Name :', threading.current_thread().getName(), 'Start', url)
    urlopen(url)
    print('Thread Name :', threading.current_thread().getName(), 'Done', url)

def main():
    with ThreadPoolExecutor(max_workers=10) as executor:
        for url in urls:
            executor.submit(fetch, url)

if __name__ == '__main__':
    # 함수 실행
    main()
    # 완료시간 - 시작시간
    duration = timeit.default_timer() - start
    # 총 실행 시간
    print('Total Time : ', duration)

Thread Name :Thread Name : ThreadPoolExecutor-0_1 Start https://google.com
 ThreadPoolExecutor-0_0 Start http://daum.net
Thread Name : ThreadPoolExecutor-0_2 Start https://apple.com
Thread Name : ThreadPoolExecutor-0_3 Start https://tistory.com
Thread Name : ThreadPoolExecutor-0_4 Start https://github.com/
Thread Name : ThreadPoolExecutor-0_5 Start https://gmarket.co.kr/
Thread Name : ThreadPoolExecutor-0_4 Done https://github.com/
Thread Name : ThreadPoolExecutor-0_3 Done https://tistory.com
Thread Name : ThreadPoolExecutor-0_2 Done https://apple.com
Thread Name : ThreadPoolExecutor-0_0 Done http://daum.net
Thread Name : ThreadPoolExecutor-0_1 Done https://google.com
Total Time :  0.48317150000002584


In [1]:
import asyncio
import timeit
from urllib.request import urlopen
from concurrent.futures import ThreadPoolExecutor
import threading

# 시작 시간
start = timeit.default_timer()
urls = ['http://daum.net', 'https://google.com', 'https://apple.com', 'https://tistory.com', 'https://github.com/']


async def fetch(url, executor):
    # 쓰레드 이름 주목!
    print('Thread Name :', threading.current_thread().getName(), 'Start', url)
    # 실행
    res = await loop.run_in_executor(executor, urlopen, url)
    print('Thread Name :', threading.current_thread().getName(), 'Done', url)
    # 반환
    return res.read()[0:5]

async def main():
    # 쓰레드 풀 생성
    executor = ThreadPoolExecutor(max_workers=10)

    # asyncio.ensure_future :
    futures = [
        asyncio.ensure_future(fetch(url, executor)) for url in urls
    ]
    
    # 결과 취합
    rst = await asyncio.gather(*futures)

    print()
    print('Result : ', rst)

if __name__ == '__main__':
    # 루프 생성
    loop = asyncio.get_event_loop() # yeild를 만나 멈추어 있으면 다른 다음 제어권을 가져옴
    # 루프 대기
    loop.run_until_complete(main())
    # 완료시간 - 시작시간
    duration = timeit.default_timer() - start
    # 총 실행 시간
    print('Total Time : ', duration)

RuntimeError: This event loop is already running

In [2]:
import asyncio
import timeit
from urllib.request import urlopen
from concurrent.futures import ThreadPoolExecutor
import threading

# 시작 시간
start = timeit.default_timer()
urls = ['http://daum.net', 'https://google.com', 'https://apple.com', 'https://tistory.com', 'https://github.com/']


async def fetch(url, executor):
    # 쓰레드 이름 주목!
    print('Thread Name :', threading.current_thread().getName(), 'Start', url)
    # 실행
    res = await loop.run_in_executor(executor, urlopen, url)
    print('Thread Name :', threading.current_thread().getName(), 'Done', url)
    # 반환
    return res.read()[0:5]

async def main():
    # 쓰레드 풀 생성
    executor = ThreadPoolExecutor(max_workers=10)

    # asyncio.ensure_future :
    futures = [
        asyncio.ensure_future(fetch(url, executor)) for url in urls
    ]
    
    # 결과 취합
    rst = await asyncio.gather(*futures)

    print()
    print('Result : ', rst)

if __name__ == '__main__':
    # 루프 생성
    loop = asyncio.get_event_loop()
    # main 코루틴 실행
    asyncio.run(main())
    # 완료시간 - 시작시간
    duration = timeit.default_timer() - start
    # 총 실행 시간
    print('Total Time : ', duration)

RuntimeError: asyncio.run() cannot be called from a running event loop